In [1]:
! git clone https://github.com/myshell-ai/OpenVoice.git

Cloning into 'OpenVoice'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 460 (delta 11), reused 7 (delta 7), pack-reused 445 (from 2)
Receiving objects: 100% (460/460), 3.85 MiB | 19.63 MiB/s, done.
Resolving deltas: 100% (218/218), done.


In [2]:
pip install -r /kaggle/working/OpenVoice/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 92.2 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.7 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 38.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pandas to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.3 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of

In [3]:
%cd OpenVoice

/kaggle/working/OpenVoice


In [4]:
!pip install -e .

Obtaining file:///kaggle/working/OpenVoice
  Preparing metadata (setup.py) ... done
  Running setup.py develop for MyShell-OpenVoice


In [5]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import BaseSpeakerTTS, ToneColorConverter

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [6]:
ckpt_base = '/kaggle/input/checkpoint/checkpoints/base_speakers/EN'
ckpt_converter = '/kaggle/input/checkpoint/checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint '/kaggle/input/checkpoint/checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []


Loaded checkpoint '/kaggle/input/checkpoint/checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [7]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

In [8]:
reference_speaker = 'add audio file path' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

OpenVoice version: v1


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


[(0.206, 20.402), (20.494, 22.77)]
after vad: dur = 22.472


/usr/local/lib/python3.11/dist-packages/torch/functional.py:709: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(  # type: ignore[attr-defined]


In [9]:
import os
import torch

# Correct directory path
output_dir = '/kaggle/working/output'
os.makedirs(output_dir, exist_ok=True)

# Load speaker embedding
source_se = torch.load(f'{ckpt_base}/en_style_se.pth').to(device)

# Correct save path and tmp path
src_path = f'{output_dir}/tmp.wav'
save_path = f'{output_dir}/whis.wav'

# Run the base speaker TTS
text = "add text that you want to clone"
# Available choices: friendly, cheerful, excited, sad, angry, terrified, shouting, whispering
base_speaker_tts.tts(text, src_path, speaker='whispering', language='English', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message
)

 > Text splitted to sentences.
Hello, I am kartik, how are you doing? I hope you all are fine.
 > ===========================
hɛˈloʊ, aɪ æm kaɹtik*, haʊ əɹ ju duɪŋ? aɪ hoʊp ju ɔɫ əɹ faɪn.
 length:61
 length:61
